In [7]:
#Python Imports
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import logging
from optparse import OptionParser
import sys
from time import time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd
import re
from pandas.io.json import json_normalize
import json
import pandas as pd
import numpy as np
from surprise import SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.knns import KNNWithZScore, KNNBaseline
from surprise.prediction_algorithms.matrix_factorization import NMF
from pandas.io.json import json_normalize
from pymongo import MongoClient
from collections import defaultdict
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import re
from sklearn.cluster import KMeans, MiniBatchKMeans
import logging
from optparse import OptionParser
import sys
from time import time
from sklearn.decomposition import TruncatedSVD
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [105]:
def convert_ids(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('int64')

def convert_to_float(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('float64')

def to_json(csv_entry):
    return json.loads(re.sub('\'', '"', csv_entry))


def get_top_n(predictions, n=200):
    '''COPIED FROM SUPRISE API
    Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def get_movie_name(movie_id):
    return ratings_with_movie_names[ratings_with_movie_names.id == movie_id]['title'].iloc[0]


def print_user_prediction(userId, predictions_dict, meta_df):
    users_viewed_movies = ratings_with_movie_names[ratings_with_movie_names['userId'] == userId][
        ['rating', 'original_title']]
    print(f'User {userId} has viewed the following movies:\n')

    for row in users_viewed_movies.itertuples():
        rating = row[1]
        original_title = row[2]
        print(f'\t{original_title}, Rating: {rating}')

    print(f'\nThe following movies are recommended for User {userId}\n')
    recommended_movies = [get_movie_name(mov_id[0], meta_df) for mov_id in predictions_dict[userId]]

    for movie in recommended_movies:
        print(f'\t{movie}')

def get_movie_name(movie_id, movie_meta_df):
    return movie_meta_df[movie_meta_df.id == movie_id]['title'].iloc[0]

def get_movie_id(title, movie_meta_df):
    return movie_meta_df[movie_meta_df.title == title]['id'].iloc[0]


def get_all_movies_in_cluster(cluster_number, cluster_dict, meta_df):
    movies = cluster_dict[cluster_number]
    return [get_movie_name(mov, meta_df) for mov in movies]

####

# cluster_distribution = [len(movies) for (clust, movies) in movie_summaries_clustered.items()]

def get_cluster_number(movie, cluster_zip):
    for cluster, movie_id in cluster_zip:

        if movie_id == movie:
            return cluster

    raise Exception('Movie not found in cluster')

In [11]:
movies_metadata_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                                 , converters={ 'id': lambda x: convert_ids(x)
                                               , 'imdb_id': lambda x: convert_ids(x)
                                               ,'popularity': lambda x: convert_to_float(x)
                                               ,'genres': lambda x: to_json(x)}
                                 , usecols=['id', 'original_title'
                                                , 'genres' #'homepage'
                                                , 'overview', 'popularity', 'poster_path'
                                                , 'release_date', 'revenue', 'runtime'
                                                , 'spoken_languages', 'title'
                                                , 'vote_average', 'vote_count']
                                , dtype={'populariy': np.float64}
                                , parse_dates=True)


movies_lookup_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'title'])
#####################################
##SVD DATA SET
movies_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'original_title', 'belongs_to_collection'
                                 , 'budget', 'genres', 'homepage'
                                 ,'imdb_id', 'overview', 'popularity', 'poster_path'
                                 , 'production_companies','release_date', 'revenue', 'runtime',
                                 'spoken_languages', 'status', 'tagline', 'title', 'video',
                                 'vote_average', 'vote_count'])
#####################################

ratings_df = pd.read_csv('../data/the-movies-dataset/ratings_small.csv')



content_filter_df = pd.read_pickle('../data/content_filter_df.pkl')
content_filter_df = content_filter_df[['id',
 'popularity',
 #'release_date',
 'vote_average',
 'release_year',
 0,1,2,3,4,5, 6, 7, 8,9,10,11,12,13,14,15,16,17,18,19]]
content_filter_df = content_filter_df.dropna()

idx = pd.Index(content_filter_df['id'])
idx
content_filter_df.index = idx

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# SVD

In [12]:
###May need Fuzzy matching, but for now:
movies_df = movies_df[movies_df.spoken_languages == """[{'iso_639_1': 'en', 'name': 'English'}]"""]

In [13]:
ratings_with_movie_names = ratings_df.merge(movies_df[['id', 'original_title']], how='left', left_on='movieId', right_on='id')
ratings_with_movie_names = ratings_with_movie_names[ratings_with_movie_names.original_title.isnull() == False]

In [14]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_with_movie_names[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

### You Can Try Different Algos Here

In [106]:
algo = SVD(verbose=True)
algo.fit(trainset)

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8911  0.8953  0.9027  0.8958  0.9052  0.8980  0.0052  
MAE (testset)     0.6891  0.6896  0.6982  0.6904  0.7003  0.6935  0.0047  
Fit time          0.84    0.83    0.81    0.81    0.80    0.82    0.02    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'test_rmse': array([0.89110445, 0.89526235, 0.90267926, 0.89580204, 0.90517404]),
 'test_mae': array([0.6890893 , 0.68955891, 0.69815064, 0.69042398, 0.70027319]),
 'fit_time': (0.8358199596405029,
  0.8303539752960205,
  0.8122599124908447,
  0.8065412044525146,
  0.7950737476348877),
 'test_time': (0.023736238479614258,
  0.023263216018676758,
  0.022314071655273438,
  0.022607088088989258,
  0.02210211753845215)}

In [107]:
predictions = algo.test(testset)
### Tune this value to get fewer results faster, but less options to choose from
top_n = get_top_n(predictions)


predicted_movies_by_name = defaultdict(list)

### This builds the dictionary of predicted movies for all users
for key, value in top_n.items():
    predicted_movies_by_name[key] = [get_movie_name(mov_id[0], movies_metadata_df) for mov_id in value]

Get a User's top 3 Movies for reference 

In [109]:
UserFavoriteRating = namedtuple('UserFavoriteRating', ['title', 'rating'])

def users_favorite_movies(n, userId, predictions_dict, meta_df):
    users_viewed_movies = ratings_with_movie_names[ratings_with_movie_names['userId'] == userId][['rating', 'original_title']]
    
    viewed_movies = []

    for row in users_viewed_movies.itertuples():
        rating = row[1]
        original_title = row[2]
        film = UserFavoriteRating(original_title, rating)
        viewed_movies.append(film)
    
    sorted(viewed_movies, key=lambda film: film[1])
    
    return viewed_movies[0:n]

## GET A USERS TOP RATED MOVIES

In [41]:
users_top_n_movies(5, 321, predicted_movies_by_name, movies_metadata_df)

[UserFavoriteRating(title='The Endless Summer', rating=5.0),
 UserFavoriteRating(title='Rumble Fish', rating=5.0),
 UserFavoriteRating(title='A River Runs Through It', rating=4.0),
 UserFavoriteRating(title='Terminator 3: Rise of the Machines', rating=5.0),
 UserFavoriteRating(title='The Poseidon Adventure', rating=5.0)]

**At this point, you should have user personas to get a pool of movies to choose from, not simply pre-made users:**

In [45]:
print_user_prediction(321, top_n, movies_metadata_df)

User 321 has viewed the following movies:

	The Endless Summer, Rating: 5.0
	Rumble Fish, Rating: 5.0
	A River Runs Through It, Rating: 4.0
	Terminator 3: Rise of the Machines, Rating: 5.0
	The Poseidon Adventure, Rating: 5.0
	Apollo 13, Rating: 4.0
	Men in Black II, Rating: 4.0
	A View to a Kill, Rating: 4.0
	Videodrome, Rating: 4.0
	The Thomas Crown Affair, Rating: 5.0
	Hollywoodland, Rating: 4.0
	Bridge to Terabithia, Rating: 4.0
	Rope, Rating: 3.0
	A Time to Kill, Rating: 4.0
	Little Buddha, Rating: 3.0
	Jungle Fever, Rating: 4.0
	Return of the Jedi, Rating: 2.0
	Don Juan DeMarco, Rating: 2.0
	Twin Peaks: Fire Walk with Me, Rating: 2.0
	My Name Is Bruce, Rating: 2.0
	The Bachelor, Rating: 3.0
	Cold Mountain, Rating: 4.0
	In the Name of the King: A Dungeon Siege Tale, Rating: 2.0
	Short Circuit, Rating: 1.0

The following movies are recommended for User 321

	Nell
	Cool Hand Luke
	Space Jam
	While You Were Sleeping
	Laura
	Sleepless in Seattle
	The Thomas Crown Affair
	Birdman of Al

In [18]:
print_user_prediction(47, top_n, movies_metadata_df)

User 47 has viewed the following movies:

	48 Hrs., Rating: 4.0
	Back to the Future Part II, Rating: 4.0
	20,000 Leagues Under the Sea, Rating: 5.0
	Muriel's Wedding, Rating: 4.0
	High Noon, Rating: 4.0
	Terminator 3: Rise of the Machines, Rating: 4.0
	Faster, Pussycat! Kill! Kill!, Rating: 4.0
	Silent Hill, Rating: 4.0
	The Conversation, Rating: 3.0
	To Kill a Mockingbird, Rating: 3.0

The following movies are recommended for User 47

	The Million Dollar Hotel
	Galaxy Quest
	Hard Target
	Point Break
	The Good Thief


In [19]:
print_user_prediction(321, top_n, movies_metadata_df)

User 321 has viewed the following movies:

	The Endless Summer, Rating: 5.0
	Rumble Fish, Rating: 5.0
	A River Runs Through It, Rating: 4.0
	Terminator 3: Rise of the Machines, Rating: 5.0
	The Poseidon Adventure, Rating: 5.0
	Apollo 13, Rating: 4.0
	Men in Black II, Rating: 4.0
	A View to a Kill, Rating: 4.0
	Videodrome, Rating: 4.0
	The Thomas Crown Affair, Rating: 5.0
	Hollywoodland, Rating: 4.0
	Bridge to Terabithia, Rating: 4.0
	Rope, Rating: 3.0
	A Time to Kill, Rating: 4.0
	Little Buddha, Rating: 3.0
	Jungle Fever, Rating: 4.0
	Return of the Jedi, Rating: 2.0
	Don Juan DeMarco, Rating: 2.0
	Twin Peaks: Fire Walk with Me, Rating: 2.0
	My Name Is Bruce, Rating: 2.0
	The Bachelor, Rating: 3.0
	Cold Mountain, Rating: 4.0
	In the Name of the King: A Dungeon Siege Tale, Rating: 2.0
	Short Circuit, Rating: 1.0

The following movies are recommended for User 321

	Nell
	Cool Hand Luke
	Space Jam
	While You Were Sleeping
	Laura


## Get Collab Filt. Recs

In [128]:
UserFavoriteRating = namedtuple('UserFavoriteRating', ['title', 'rating'])
def collab_filter_recommendations(user, top_ns, movie_meta_df):
    
    predictions = top_ns[user]
    
    return [UserFavoriteRating(get_movie_name(pred[0], movie_meta_df), pred[1]) for pred in predictions]

        


In [129]:
collab_filter_recommendations(1, top_n, movies_metadata_df)

[UserFavoriteRating(title='Galaxy Quest', rating=4.289329338562095),
 UserFavoriteRating(title='My Darling Clementine', rating=4.1546576614959205),
 UserFavoriteRating(title='The 39 Steps', rating=4.082967613931596),
 UserFavoriteRating(title='Point Break', rating=4.073328904018292),
 UserFavoriteRating(title='While You Were Sleeping', rating=4.053242654989873),
 UserFavoriteRating(title='Hard Target', rating=4.050979529214875),
 UserFavoriteRating(title='Edward Scissorhands', rating=4.036904094491688),
 UserFavoriteRating(title='Confession of a Child of the Century', rating=4.018606999511867),
 UserFavoriteRating(title='The Million Dollar Hotel', rating=4.01539089870482),
 UserFavoriteRating(title='Straw Dogs', rating=4.015042430679403),
 UserFavoriteRating(title='Sleepless in Seattle', rating=4.010680244097053),
 UserFavoriteRating(title='The Thomas Crown Affair', rating=4.007859007001082),
 UserFavoriteRating(title='Rumble Fish', rating=3.99179440329313),
 UserFavoriteRating(title='

# Content Filter

In [20]:
def top_n_closest_content_filtered(n, movie_id, content_filter_df):
    similarity_arr = cosine_similarity(np.array(content_filter_df.loc[movie_id]).reshape(1, -1), content_filter_df)[0].tolist()
    tups = [(similarity_arr.index(value), value) for value in similarity_arr]
    tups = sorted(tups, key=lambda x: x[1], reverse=True)
    return tups[0:n]

In [32]:
from collections import namedtuple

ContentFilterRecommendation = namedtuple('ContentFilterRecommendation', ['id_origin', 'title'
                                 ,'id_2', 'title_2'
                                ,'similarity'])

In [97]:
def top_filtered_to_movie_rec_tuples(content_filtered_rec_list, origin_movie_id, movie_meta_data_df):
    failed = 0
    recs = []
    origin_film_name = get_movie_name(origin_movie_id, movie_meta_data_df)
    
    for rec in content_filtered_rec_list:
        try:

            film_2_id = rec[0]
            film_2_name = get_movie_name(rec[0], movie_meta_data_df)
            sim_score = rec[1]

            recs.append(ContentFilterRecommendation(origin_movie_id, origin_film_name
                                                   , film_2_id, film_2_name 
                                                    , sim_score))
        except:
            failed +=1
            
            pass
    print(f'lookup failed {failed} times.')
    return recs
    

In [34]:
sleepless_filtered_recs = top_n_closest_content_filtered(20, 858, content_filter_df)
sleepless_filtered_recs

[(534, 0.9999999999999998),
 (1904, 0.9999985798898556),
 (1208, 0.9999981736066476),
 (4137, 0.9999979433200008),
 (2681, 0.9999972682954646),
 (2792, 0.9999971992504212),
 (362, 0.999996301872156),
 (8232, 0.999996033283064),
 (984, 0.9999959498257401),
 (10833, 0.9999958404012185),
 (4870, 0.9999958239470732),
 (6816, 0.999995472066687),
 (2863, 0.9999946890182448),
 (2070, 0.9999937776328239),
 (4086, 0.9999934671526973),
 (2989, 0.999993301817415),
 (2077, 0.9999926890765405),
 (4899, 0.9999924966707522),
 (6451, 0.999992266144631),
 (535, 0.9999920723155129)]

In [35]:
videodrome_recs = top_n_closest_content_filtered(100, 837, content_filter_df)
videodrome_recs

[(6614, 0.9999999999999998),
 (1198, 0.9999995097028028),
 (4008, 0.9999991389263577),
 (9200, 0.9999989572521445),
 (6953, 0.9999986910447686),
 (1905, 0.9999980671834254),
 (9537, 0.9999976013364797),
 (1248, 0.9999975103862115),
 (1201, 0.9999971499357722),
 (1793, 0.9999969306607781),
 (3237, 0.9999965440703428),
 (3343, 0.9999957554021307),
 (1200, 0.9999957552508402),
 (6617, 0.9999950239778762),
 (7761, 0.999993762261875),
 (2077, 0.9999936099151473),
 (8659, 0.9999932524968942),
 (12978, 0.9999927727129296),
 (4086, 0.9999924818697025),
 (6451, 0.999991177576172),
 (4870, 0.9999893304264468),
 (14377, 0.9999886797873373),
 (4137, 0.9999859749819323),
 (4899, 0.999984350412921),
 (2681, 0.9999838953145571),
 (10698, 0.9999826508811969),
 (362, 0.999981562905407),
 (534, 0.999973728971039),
 (1208, 0.9999688226084048),
 (2322, 0.9999661617492284),
 (2863, 0.9999651504414468),
 (1912, 0.9999626845604357),
 (1904, 0.9999624868507314),
 (7625, 0.9999601337332559),
 (882, 0.999958196

In [36]:
top_filtered_to_movie_rec_tuples(videodrome_recs, 837, movies_lookup_df)

lookup failed 6614
lookup failed 1198
lookup failed 4008
lookup failed 9200
lookup failed 6953
lookup failed 1905
lookup failed 9537
lookup failed 1201
lookup failed 3237
lookup failed 3343
lookup failed 1200
lookup failed 6617
lookup failed 7761
lookup failed 8659
lookup failed 12978
lookup failed 4086
lookup failed 6451
lookup failed 4870
lookup failed 14377
lookup failed 4137
lookup failed 4899
lookup failed 2681
lookup failed 10698
lookup failed 362
lookup failed 1208
lookup failed 2863
lookup failed 7625
lookup failed 882
lookup failed 3259
lookup failed 8232
lookup failed 2792
lookup failed 5266
lookup failed 6816
lookup failed 2070
lookup failed 0
lookup failed 7692
lookup failed 4504
lookup failed 3567
lookup failed 2606
lookup failed 5337
lookup failed 2560
lookup failed 33804
lookup failed 25819
lookup failed 32106
lookup failed 2803
lookup failed 7462
lookup failed 7163
lookup failed 6822
lookup failed 46
lookup failed 7282
lookup failed 7706
lookup failed 3320
lookup failed

[ContentFilterRecommendation(id_origin=837, title='Videodrome', id_2=1248, title_2='Hannibal Rising', similarity=0.9999975103862115),
 ContentFilterRecommendation(id_origin=837, title='Videodrome', id_2=1793, title_2='Max Dugan Returns', similarity=0.9999969306607781),
 ContentFilterRecommendation(id_origin=837, title='Videodrome', id_2=2077, title_2='The Rink', similarity=0.9999936099151473),
 ContentFilterRecommendation(id_origin=837, title='Videodrome', id_2=534, title_2='Terminator Salvation', similarity=0.999973728971039),
 ContentFilterRecommendation(id_origin=837, title='Videodrome', id_2=2322, title_2='Sneakers', similarity=0.9999661617492284),
 ContentFilterRecommendation(id_origin=837, title='Videodrome', id_2=1912, title_2='Reclaim Your Brain', similarity=0.9999626845604357),
 ContentFilterRecommendation(id_origin=837, title='Videodrome', id_2=1904, title_2='Memoirs of a Geisha', similarity=0.9999624868507314),
 ContentFilterRecommendation(id_origin=837, title='Videodrome', 

In [31]:
top_filtered_to_movie_rec_tuples(sleepless_filtered_recs, 858, movies_lookup_df)

lookup failed 1208
lookup failed 4137
lookup failed 2681
lookup failed 2792
lookup failed 362
lookup failed 8232
lookup failed 4870
lookup failed 6816
lookup failed 2863
lookup failed 2070
lookup failed 4086
lookup failed 4899
lookup failed 6451


[MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=534, title_2='Terminator Salvation', similarity=0.9999999999999998),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=1904, title_2='Memoirs of a Geisha', similarity=0.9999985798898556),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=984, title_2='Dirty Harry', similarity=0.9999959498257401),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=10833, title_2='Heartbreakers', similarity=0.9999958404012185),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=2989, title_2='Roar', similarity=0.999993301817415),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=2077, title_2='The Rink', similarity=0.9999926890765405),
 MovieRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=535, title_2='Flashdance', similarity=0.9999920723155129)]

# Graph Filter

In [46]:
user_number = 321
predictions_dict = predicted_movies_by_name
get_graph_on = users_top_n_movies(5, user_number, predictions_dict, movies_metadata_df)

In [47]:
get_graph_on

[UserFavoriteRating(title='The Endless Summer', rating=5.0),
 UserFavoriteRating(title='Rumble Fish', rating=5.0),
 UserFavoriteRating(title='A River Runs Through It', rating=4.0),
 UserFavoriteRating(title='Terminator 3: Rise of the Machines', rating=5.0),
 UserFavoriteRating(title='The Poseidon Adventure', rating=5.0)]

In [92]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=('neo4j', 'n;slr#!21'))


def get_count_first_degree_films_of(tx, title):
    for record in tx.run("MATCH (origin:MOVIE)-[:APPEARED_IN]-(actor)-[:APPEARED_IN]-(first_movie:MOVIE)"
                         "WHERE origin.title = {title} "
                         "RETURN count(*)", title=title):
        print(record[0])
        
def get_first_degree_films_of(tx, title):
    nodes = []
    
    for record in tx.run("MATCH (origin:MOVIE)-[:APPEARED_IN]-(actor)-[:APPEARED_IN]-(first_movie:MOVIE)"
                         "WHERE origin.title = {title} "
                         "RETURN first_movie", title=title):
        
        nodes.append(record.data())
    
    return nodes

first_degree_away = None
with driver.session() as session:
    first_degree_away = session.read_transaction(get_first_degree_films_of, 'Sleepless in Seattle')

In [77]:
first_degree_away[0]

{'first_movie': <Node id=247078 labels={'MOVIE'} properties={'title': 'Brother Nature', 'movie_id': 292834}>}

In [80]:
dir(first_degree_away[0]['first_movie'])

['__abstractmethods__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_graph',
 '_id',
 '_labels',
 '_properties',
 '_update',
 'get',
 'graph',
 'id',
 'items',
 'keys',
 'labels',
 'values']

In [85]:
first_degree_away[0]['first_movie'].get('title')

'Brother Nature'

In [86]:
first_degree_away[0]['first_movie'].get('movie_id')

292834

In [94]:
GraphMember = namedtuple('GraphMember', ['title', 'movie_id'])

def neo4j_results_to_tuples(results):
    return [GraphMember(node['first_movie'].get('title'), node['first_movie'].get('movie_id')) for node in results]

In [95]:
neo4j_results_to_tuples(first_degree_away)

[GraphMember(title='Brother Nature', movie_id=292834),
 GraphMember(title='Everything Is Copy', movie_id=356325),
 GraphMember(title='My Big Fat Greek Wedding 2', movie_id=302688),
 GraphMember(title='Stranded', movie_id=299145),
 GraphMember(title='The Art of Getting By', movie_id=64678),
 GraphMember(title='Larry Crowne', movie_id=59861),
 GraphMember(title="It's Complicated", movie_id=22897),
 GraphMember(title='Old Dogs', movie_id=22949),
 GraphMember(title='If These Walls Could Talk', movie_id=35203),
 GraphMember(title='Beautiful Ohio', movie_id=18208),
 GraphMember(title='The Chumscrubber', movie_id=13919),
 GraphMember(title='Raise Your Voice', movie_id=14024),
 GraphMember(title='Auto Focus', movie_id=14112),
 GraphMember(title="Cheech & Chong's Next Movie", movie_id=13612),
 GraphMember(title='Volunteers', movie_id=19259),
 GraphMember(title='The Bonfire of the Vanities', movie_id=9586),
 GraphMember(title='The Story of Us', movie_id=12220),
 GraphMember(title='Runaway Bride'

### Hybridization

In [99]:
top_filtered_to_movie_rec_tuples(sleepless_filtered_recs, 858, movies_lookup_df)[0:2]

lookup failed 13 times.


[ContentFilterRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=534, title_2='Terminator Salvation', similarity=0.9999999999999998),
 ContentFilterRecommendation(id_origin=858, title='Sleepless in Seattle', id_2=1904, title_2='Memoirs of a Geisha', similarity=0.9999985798898556)]

In [100]:
neo4j_results_to_tuples(first_degree_away)[0:2]

[GraphMember(title='Brother Nature', movie_id=292834),
 GraphMember(title='Everything Is Copy', movie_id=356325)]

In [101]:
users_favorite_movies(5, 321, predicted_movies_by_name, movies_metadata_df)

[UserFavoriteRating(title='The Endless Summer', rating=5.0),
 UserFavoriteRating(title='Rumble Fish', rating=5.0),
 UserFavoriteRating(title='A River Runs Through It', rating=4.0),
 UserFavoriteRating(title='Terminator 3: Rise of the Machines', rating=5.0),
 UserFavoriteRating(title='The Poseidon Adventure', rating=5.0)]

### Run a test with a user

Filter a users top collaborative filtered suggestions through the graph:

In [135]:
sorted(users_favorite_movies(200, 321, predicted_movies_by_name, movies_metadata_df), key=lambda k: k[1], reverse=True)

[UserFavoriteRating(title='The Endless Summer', rating=5.0),
 UserFavoriteRating(title='Rumble Fish', rating=5.0),
 UserFavoriteRating(title='Terminator 3: Rise of the Machines', rating=5.0),
 UserFavoriteRating(title='The Poseidon Adventure', rating=5.0),
 UserFavoriteRating(title='The Thomas Crown Affair', rating=5.0),
 UserFavoriteRating(title='A River Runs Through It', rating=4.0),
 UserFavoriteRating(title='Apollo 13', rating=4.0),
 UserFavoriteRating(title='Men in Black II', rating=4.0),
 UserFavoriteRating(title='A View to a Kill', rating=4.0),
 UserFavoriteRating(title='Videodrome', rating=4.0),
 UserFavoriteRating(title='Hollywoodland', rating=4.0),
 UserFavoriteRating(title='Bridge to Terabithia', rating=4.0),
 UserFavoriteRating(title='A Time to Kill', rating=4.0),
 UserFavoriteRating(title='Jungle Fever', rating=4.0),
 UserFavoriteRating(title='Cold Mountain', rating=4.0),
 UserFavoriteRating(title='Rope', rating=3.0),
 UserFavoriteRating(title='Little Buddha', rating=3.0),

In [130]:
## Get a graph for the top 3 movies in their library

In [158]:
def get_top_three_favs(user_id):
    
    favorite_seen_movie_array = users_favorite_movies(200, user_id, predicted_movies_by_name, movies_metadata_df)
    sorted_seen_movies = sorted(favorite_seen_movie_array, key=lambda k: k[1], reverse=True)
    return sorted_seen_movies[0:3]

#### User Favorites

In [159]:
get_top_three_favs(321)

[UserFavoriteRating(title='The Endless Summer', rating=5.0),
 UserFavoriteRating(title='Rumble Fish', rating=5.0),
 UserFavoriteRating(title='Terminator 3: Rise of the Machines', rating=5.0)]

#### Collab Filter List

In [196]:
collab_filtered_movies = sorted(collab_filter_recommendations(321, top_n, movies_metadata_df), key=lambda k: k[1], reverse=True)
collab_filtered_movies

[UserFavoriteRating(title='Galaxy Quest', rating=4.334533896704643),
 UserFavoriteRating(title='Sleepless in Seattle', rating=4.27342183921424),
 UserFavoriteRating(title='Lonely Hearts', rating=4.223099189462814),
 UserFavoriteRating(title='Dawn of the Dead', rating=4.202043116624332),
 UserFavoriteRating(title='Frankenstein', rating=4.1703237080426865),
 UserFavoriteRating(title='Cool Hand Luke', rating=4.152970819654721),
 UserFavoriteRating(title='The Good Thief', rating=4.149608475539469),
 UserFavoriteRating(title='The Million Dollar Hotel', rating=4.12513991020092),
 UserFavoriteRating(title='Murder She Said', rating=4.123267647679061),
 UserFavoriteRating(title='Laura', rating=4.122503537242561),
 UserFavoriteRating(title='Beverly Hills Cop III', rating=4.121999124612101),
 UserFavoriteRating(title='Edward Scissorhands', rating=4.121874651824622),
 UserFavoriteRating(title='Nell', rating=4.121102922521525),
 UserFavoriteRating(title='The 39 Steps', rating=4.118446081809196),
 U

#### Content Filtered

In [194]:
#get_movie_id('The Endless Summer', movies_metadata_df)
content_filtered_tups = top_n_closest_content_filtered(50, 321, content_filter_df)
content_filtered_tups

[(6609, 1.0),
 (385, 0.9999992491593038),
 (4010, 0.9999984682726184),
 (13305, 0.9999965660482049),
 (7589, 0.9999960412941354),
 (11109, 0.9999946994256279),
 (5160, 0.9999946149750253),
 (1777, 0.9999918858008893),
 (4501, 0.9999908559184394),
 (5244, 0.9999905984056134),
 (6710, 0.9999875332197931),
 (1467, 0.9999871317963381),
 (4499, 0.9999865321641036),
 (475, 0.9999837002008604),
 (2239, 0.999982874621271),
 (6536, 0.9999821365685515),
 (896, 0.9999819395820446),
 (11403, 0.9999816424074247),
 (7928, 0.9999800105069793),
 (10234, 0.9999766408432524),
 (2597, 0.9999737775532492),
 (550, 0.9999737094180704),
 (3035, 0.9999723994014073),
 (4191, 0.9999696320331022),
 (4814, 0.9999684667449661),
 (415, 0.9999676944573901),
 (7135, 0.9999655181174771),
 (6217, 0.9999642660388708),
 (7159, 0.9999578672417527),
 (10328, 0.9999550268383395),
 (9635, 0.9999482645244442),
 (1227, 0.9999478813916667),
 (7486, 0.9999438829654383),
 (6378, 0.9999400951025329),
 (11259, 0.9999373998993288),


In [179]:
favorite_list = get_top_three_favs(321)
movie_names = [movie[0] for movie in favorite_list]

In [187]:
def get_connected_movies(list_favorite_movies):
    return list_connected_movies

first_degree_away = []
with driver.session() as session:
    
    for movie in movie_names:
        first_degree_away_films = session.read_transaction(get_first_degree_films_of, movie)
        film_tups = neo4j_results_to_tuples(first_degree_away_films)
        first_degree_away.extend(film_tups)

In [188]:
first_degree_away

[GraphMember(title='Step Into Liquid', movie_id=15292),
 GraphMember(title='The Search for Freedom', movie_id=353125),
 GraphMember(title='Step Into Liquid', movie_id=15292),
 GraphMember(title='On Any Sunday', movie_id=29251),
 GraphMember(title='Never So Few', movie_id=4820),
 GraphMember(title='Seven', movie_id=102961),
 GraphMember(title='Chrome and Hot Leather', movie_id=69150),
 GraphMember(title='The Swinging Barmaids', movie_id=28757),
 GraphMember(title='Bulletproof', movie_id=87533),
 GraphMember(title='Angels Die Hard', movie_id=42533),
 GraphMember(title='Platoon Leader', movie_id=48457),
 GraphMember(title='C.C. and Company', movie_id=50372),
 GraphMember(title='The Ultimate Warrior', movie_id=32053),
 GraphMember(title='Sweet Jesus, Preacherman', movie_id=94248),
 GraphMember(title='Hammer', movie_id=90728),
 GraphMember(title='Hollywood Man', movie_id=29860),
 GraphMember(title='Mail Order Bride', movie_id=202941),
 GraphMember(title='The Last American Hero', movie_id=31

In [190]:
graph_id_content_filter = set({movie[1] for movie in first_degree_away})

In [192]:
graph_name_collab_filter = set({movie[0] for movie in first_degree_away})

In [195]:
content_id_set = set(movie[0] for movie in content_filtered_tups)

In [197]:
collab_movie_name_set = set(movie[0] for movie in collab_filtered_movies)

In [198]:
graph_name_collab_filter & collab_movie_name_set

{'Armed and Dangerous',
 'Cool Hand Luke',
 'Ghost Rider',
 "One Night at McCool's",
 'Pale Rider',
 'Raw Deal',
 'The Ghost of Frankenstein',
 "Vampire's Kiss"}

In [199]:
content_id_set & graph_id_content_filter

{415, 550}

In [201]:
get_movie_name(550, movies_metadata_df)

'Fight Club'